<a href="https://colab.research.google.com/github/Mohamad-Alshalabi/Coursera_Capstone/blob/master/DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**THE BATTLE OF NEIGHBORHOODS (WEEK1)** 
##**DATA**



#**Data**

**The data is obtained from various sources. The issue of data availability is overcome by creating our own sources.** 
  
To answer the question of which neighborhood in Stockholm has the least coverage of food venues, we should look into two main variables: the number of venues in each neighborhood. The Second variable is concerning the number of inhabitants in the neighborhood. 
In the list below I include the three source of data used in this study.     
1. The name and the postal code for each neighborhood collected and scrapped from the following website: https://worldpostalcode.com
2. The Latitude, the Longitude and the Population for each neighborhood in stockholm manually collected and gathered in one CSV file.
3. Name, location, latitude, longitude for each food venue in the neighborhoods collected from https://api.foursquare.com website.


##**DATA EXPLORATION**

In [1]:
!pip install geocoder
!pip install folium
import numpy as np 
import pandas as pd 
import geocoder 
import requests 
from requests import get 
from bs4 import BeautifulSoup 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
print("Importing is done")

Importing is done


In [2]:
url = "https://worldpostalcode.com/sweden/stockholm/"
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [3]:
Stockholm_containers = html_soup.find_all('div', class_ = 'container')
print(type(Stockholm_containers))
print(len(Stockholm_containers))

<class 'bs4.element.ResultSet'>
64


In [4]:
names = []
for container in Stockholm_containers:
    if container.find('div') is not None:
        name = container.div.text
    names.append(name)

In [5]:
Stockholm_codes = html_soup.find_all( class_ = 'code')
codes = []
for code in Stockholm_codes :
    i = Stockholm_codes.index(code)
    code = str(code)
    code = code.replace('<div class="code"><span>', '')
    code = code.replace('</span></div>', '')    
    code = code.split('</span> <span>') 
    codes.append(code)

In [6]:
data = {'Neighborhood': names, 'Postal_Code': codes}
Data= pd.DataFrame(data= data, columns= ['Neighborhood','Postal_Code'])

In [7]:
Stockholm_data = pd.read_csv("https://raw.githubusercontent.com/Mohamad-Alshalabi/Coursera_Capstone/master/Book2.csv")

Stockholm_data.head()

,Neighborhood,Latitude,Longitude,Population
0,Akersberga,59.4803,18.3108,28033
1,Alta,59.2555,18.1781,9989
2,Alvsjo,59.2745,18.0051,1530
3,Arsta,59.2986,18.0459,16807
4,Bagarmossen,59.2744,18.1338,10915


In [8]:
Stockholm_geo_data = pd.merge(Stockholm_data, Data, on='Neighborhood')
Stockholm_geo_data.head()

,Neighborhood,Latitude,Longitude,Population,Postal_Code
0,Akersberga,59.4803,18.3108,28033,[184 20]
1,Alta,59.2555,18.1781,9989,[138 20]
2,Alvsjo,59.2745,18.0051,1530,"[125 00, 125 58, 125 52, 125 53, 125 54, 125 5..."
3,Arsta,59.2986,18.0459,16807,"[120 02, 120 52, 120 59, 120 58, 120 57, 120 5..."
4,Bagarmossen,59.2744,18.1338,10915,"[128 04, 128 05, 128 41, 128 42, 128 43, 128 4..."


In [9]:
print('The dataframe has {} neighborhoods.'.format( len(Stockholm_geo_data['Neighborhood'].unique())))

The dataframe has 64 neighborhoods.


In [10]:
address = 'Stockholm Sweden'
geolocator = Nominatim(user_agent="stock_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.


In [11]:
map_Stockholm = folium.Map(location=[latitude,longitude], zoom_start= 10)
for lat, lng, label in zip(Stockholm_geo_data['Latitude'], Stockholm_geo_data['Longitude'], Stockholm_geo_data['Neighborhood']):
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker(
        [lat,lng],
        radius=6,
        popup= label, 
        color= 'blue',
        fill= True,
        fill_color= 'FFFF5c', 
        fill_opacity= 0.7
        ).add_to(map_Stockholm)

map_Stockholm

In [12]:
CLIENT_ID = 'CAH33GB3T5AA33ZLAPKRSPMN0HPJAF0CRI1ES31CMFKIMRT0' 
CLIENT_SECRET = 'Q1LRHG2SDJMSACGJGKA1K2CMDWGMPJQSN05CWAMEOOEFH0YL' 
VERSION = '20210217'

In [13]:
def getNearbyVenues1(names, latitudes, longitudes, radius=500, categoryIds=''):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
       
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
       
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
      
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
                  

In [14]:
Stockholm_food_venues = getNearbyVenues1(names=Stockholm_geo_data['Neighborhood'],
                                   latitudes=Stockholm_geo_data['Latitude'],
                                   longitudes=Stockholm_geo_data['Longitude'], radius=1000, categoryIds= '4d4b7105d754a06374d81259')

Akersberga
Alta
Alvsjo
Arsta
Bagarmossen
Bandhagen
Bro
Bromma
Danderyd
Ekero
Enskede
Enskede Gard
Enskededalen
Farsta
Gustavsberg
Hagersten
Hallstavik
Handen
Haninge
Hasselby
Huddinge
Jarfalla
Jarna
Johanneshov
Kista
Kungsangen
Lidingo
Marsta
Nacka
Norrtalje
Norsborg
Nykvarn
Nynashamn
Osmo
Rimbo
Ronninge
Rosersberg
Saltsjo-Boo
Saltsjobaden
Sigtuna
Skarholmen
Skarpnack
Skogas
Skondal
Sodertalje
Sollentuna
Solna
Spanga
Stockholm
Stockholm-Arlanda
Stockholm-Globen
Sundbyberg
Taby
Tomteboda
Tullinge
Tumba
Tyreso
Upplands Vasby
Vallentuna
Vallingby
Varby
Varmdo
Vasterhaninge
Vaxholm


In [15]:
Stockholm_food_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akersberga,59.4803,18.3108,ICA Kvantum Åkersberga,59.479721,18.297570,Grocery Store
1,Akersberga,59.4803,18.3108,Berga Sushi,59.482627,18.302635,Sushi Restaurant
2,Akersberga,59.4803,18.3108,The Public,59.480178,18.298988,Gastropub
3,Akersberga,59.4803,18.3108,Åkersberga (L),59.479164,18.299074,Light Rail Station
4,Akersberga,59.4803,18.3108,Apotek Hjärtat,59.479528,18.300780,Pharmacy


In [16]:
Stockholm_food_venues.shape

(1311, 7)

In [17]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [18]:
map_Stockholm_food_venues = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Stockholm_food_venues, 'red', map_Stockholm_food_venues)

map_Stockholm_food_venues

In [19]:
Stockholm_grouped_1=  Stockholm_food_venues.groupby('Neighborhood').count()
Stockholm_grouped_1.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Akersberga,12,12,12,12,12,12
Alta,9,9,9,9,9,9
Alvsjo,26,26,26,26,26,26
Arsta,37,37,37,37,37,37
Bagarmossen,21,21,21,21,21,21


In [20]:
Stockholm_grouped_1.shape

(64, 6)

In [21]:
Data_1= pd.DataFrame(data=Stockholm_grouped_1, columns=['Venue'] )
Data_1

,Venue
Neighborhood,
Akersberga,12
Alta,9
Alvsjo,26
Arsta,37
Bagarmossen,21
...,...
Vallingby,33
Varby,9
Varmdo,3


In [22]:
Stockholm= pd.merge(Stockholm_geo_data , Data_1, on='Neighborhood')

In [23]:
Stockholm

,Neighborhood,Latitude,Longitude,Population,Postal_Code,Venue
0,Akersberga,59.4803,18.3108,28033,[184 20],12
1,Alta,59.2555,18.1781,9989,[138 20],9
2,Alvsjo,59.2745,18.0051,1530,"[125 00, 125 58, 125 52, 125 53, 125 54, 125 5...",26
3,Arsta,59.2986,18.0459,16807,"[120 02, 120 52, 120 59, 120 58, 120 57, 120 5...",37
4,Bagarmossen,59.2744,18.1338,10915,"[128 04, 128 05, 128 41, 128 42, 128 43, 128 4...",21
...,...,...,...,...,...,...
59,Vallingby,59.3671,17.8691,82500,"[162 00, 162 60, 162 01, 162 88, 162 83, 162 7...",33
60,Varby,59.2635,17.8876,10727,"[143 46, 143 20]",9
61,Varmdo,59.2846,18.5208,45393,"[139 00, 139 20, 139 31]",3
62,Vasterhaninge,59.1216,18.0973,15134,"[137 20, 137 42]",11


**END**
  